In [ ]:
from utils import load_and_prepare_data, structured_prune_hidden, benchmark_cpu
from neural_network import *
import time
from sklearn.metrics import classification_report

numerical_cols = [
        "duration",
        "dst_bytes",
        "missed_bytes",
        "src_bytes",
        "src_ip_bytes",
        "src_pkts",
        "dst_pkts",
        "dst_ip_bytes",
        "http_request_body_len",
        "http_response_body_len"

    ]

categorical_cols = [
        "proto",
        "conn_state",
        "http_status_code",
        "http_method",
        "http_orig_mime_types",
        "http_resp_mime_types",
    ]


target_col = 'type'
num_target_classes = 8
dataset_path = 'datasets/http_ton.csv'
batch_size = 2048
epochs = 20
values_to_remove = {'type': ['mitm', 'dos']}

In [ ]:
train_dataloader, valid_dataloader, test_dataloader, cat_cardinalities, cw, target_names = load_and_prepare_data(
    file_path=dataset_path,
    target_col=target_col,
    numerical_cols=numerical_cols,
    categorical_cols=categorical_cols,
    batch_size=batch_size,
    rows_to_remove=values_to_remove
)

embedding_dims = [min(50, (card + 1) // 2) for card in cat_cardinalities]

In [3]:
model = NeuralNetwork(
    hidden_layers_sizes=[256, 256, 256], 
    cat_cardinalities=cat_cardinalities,
    embedding_dims=embedding_dims,
    num_numerical_features=len(numerical_cols),
    num_target_classes=num_target_classes,
)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.85, patience=5)

In [ ]:
model.fit(
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    device=device,
    optimizer=optimizer,
    lr_scheduler=scheduler,
    epochs=epochs,
    weights=cw
)

--- Epoch: 0  |  Loss: 0.3425  |  F1 Score: 0.5283  |  Accuracy: 0.8693 ---
--- Epoch: 1  |  Loss: 0.2043  |  F1 Score: 0.7053  |  Accuracy: 0.9428 ---
--- Epoch: 2  |  Loss: 0.1681  |  F1 Score: 0.7161  |  Accuracy: 0.9570 ---
--- Epoch: 3  |  Loss: 0.1360  |  F1 Score: 0.6809  |  Accuracy: 0.9630 ---
--- Epoch: 4  |  Loss: 0.1198  |  F1 Score: 0.6890  |  Accuracy: 0.9655 ---
--- Epoch: 5  |  Loss: 0.1250  |  F1 Score: 0.6973  |  Accuracy: 0.9661 ---
--- Epoch: 6  |  Loss: 0.1338  |  F1 Score: 0.7289  |  Accuracy: 0.9527 ---
--- Epoch: 7  |  Loss: 0.2285  |  F1 Score: 0.6551  |  Accuracy: 0.9234 ---
--- Epoch: 8  |  Loss: 0.1041  |  F1 Score: 0.7385  |  Accuracy: 0.9691 ---
--- Epoch: 9  |  Loss: 0.1003  |  F1 Score: 0.7602  |  Accuracy: 0.9761 ---
--- Epoch: 10  |  Loss: 0.1270  |  F1 Score: 0.7446  |  Accuracy: 0.9695 ---
--- Epoch: 11  |  Loss: 0.1185  |  F1 Score: 0.7084  |  Accuracy: 0.9762 ---
--- Epoch: 12  |  Loss: 0.1097  |  F1 Score: 0.7669  |  Accuracy: 0.9807 ---
--- Epoch

In [ ]:
pruned = structured_prune_hidden(model, keep_ratio=0.7, device="cpu")
pruned.fit(train_dataloader, valid_dataloader, weights=cw, device="cpu", epochs=10, lr=5e-4)

--- Epoch: 0  |  Loss: 0.1787  |  F1 Score: 0.7201  |  Accuracy: 0.9847 ---
--- Epoch: 1  |  Loss: 0.1715  |  F1 Score: 0.7241  |  Accuracy: 0.9858 ---
--- Epoch: 2  |  Loss: 0.1897  |  F1 Score: 0.7086  |  Accuracy: 0.9817 ---
--- Epoch: 3  |  Loss: 0.1983  |  F1 Score: 0.7259  |  Accuracy: 0.9878 ---
--- Epoch: 4  |  Loss: 0.1946  |  F1 Score: 0.7312  |  Accuracy: 0.9897 ---


In [9]:
std_times = benchmark_cpu(model, test_dataloader, num_threads=1)
print(f"Standard Model:\n {std_times}")
print("--------------------")
pruned_times = benchmark_cpu(pruned, test_dataloader, num_threads=1)
print(f"Pruned model:\n {pruned_times}")

Standard Model:
 {'batch_size': 2048, 'num_threads': 1, 'median_ms': 7.435993499711913, 'p95_ms': 9.527142450406243, 'throughput_sps': 271973.93703453563}
--------------------
Pruned model:
 {'batch_size': 2048, 'num_threads': 1, 'median_ms': 4.158681998887914, 'p95_ms': 5.349523000404588, 'throughput_sps': 491456.33222593716}


In [15]:
std = std_times['throughput_sps']
pru = pruned_times['throughput_sps']

latency_reduction_pct = (1 - std / pru) * 100
print(f"Riduzione tempo di inferenza: -{latency_reduction_pct:.1f}%")

Riduzione tempo di inferenza: -44.7%


In [16]:
y_true = torch.cat([y for _, _, y in test_dataloader]).numpy()

In [17]:
model_preds = model.predict(test_dataloader,device)
pruned_preds = pruned.predict(test_dataloader,device)

In [19]:
print("\n=== Classification Report DNN===")
print(classification_report(y_true, model_preds.numpy(), target_names=target_names, digits=4))
print("\n=== Classification Report Pruned===")
print(classification_report(y_true, pruned_preds.numpy(), target_names=target_names, digits=4))



=== Classification Report DNN===
              precision    recall  f1-score   support

        ddos     0.9745    0.9855    0.9800     50614
         dos     0.2576    0.6538    0.3696        26
   injection     0.9637    0.9372    0.9502     50968
        mitm     0.0000    0.0000    0.0000         7
      normal     0.9288    0.8427    0.8836      9197
    password     0.9841    0.9938    0.9889    189474
    scanning     0.9938    0.9189    0.9549      4686
         xss     0.9875    0.9882    0.9878    211140

    accuracy                         0.9817    516112
   macro avg     0.7612    0.7900    0.7644    516112
weighted avg     0.9816    0.9817    0.9815    516112


=== Classification Report Pruned===
              precision    recall  f1-score   support

        ddos     0.9921    0.9853    0.9887     50614
         dos     0.0000    0.0000    0.0000        26
   injection     0.9705    0.9713    0.9709     50968
        mitm     0.0000    0.0000    0.0000         7
      n

/home/mat-rib/Documents/tabular_dnn/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mat-rib/Documents/tabular_dnn/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/mat-rib/Documents/tabular_dnn/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod